In [1]:
!pip install pandas geopandas matplotlib faker pyspark pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 4.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 4.2 MB/s eta 0:00:0000:0100:01
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:

In [2]:
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, sum, max, min, avg, count, round, lit, year, month, dayofmonth, trim, hour, minute, second, dayofweek
from pyspark.sql.types import StructType, StringType, DoubleType, IntegerType, TimestampType, FloatType
from datetime import datetime
import os 
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Crear sesión de Spark
spark = SparkSession \
    .builder \
    .appName("SparkStreamingFromSocket") \
    .master("local[*]") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

# Definir el esquema para los datos JSON que se recibirán
schema = StructType() \
    .add("latitude", DoubleType()) \
    .add("longitude", DoubleType()) \
    .add("date", TimestampType()) \
    .add("customer_id", StringType()) \
    .add("employee_id", StringType()) \
    .add("quantity_products", IntegerType()) \
    .add("order_id", StringType()) \
    .add("commune_code", StringType()) \
    .add("commune_name", StringType()) \
    .add("customer_name", StringType()) \
    .add("employee_name", StringType()) \
    .add("employee_commission", DoubleType())

# Leer datos desde el socket
streaming_df = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 12345) \
    .load()

# Parsear los datos JSON utilizando el esquema definido
parsed_df = streaming_df \
    .select(from_json(col("value").cast("string"), schema).alias("parsed_value")) \
    .select("parsed_value.*")

# Función para guardar los datos recibidos en bronze
def process_data(df, epoch_id):
    try:
        hdfs_path = "/user/root/bronze"
        df.write \
          .format("parquet") \
          .mode("append") \
          .save(hdfs_path)
        df.show(truncate=False)
    except Exception as e:
        print(f"Error al procesar los datos: {e}")

# Escribir los resultados en la consola
query = parsed_df \
    .writeStream \
    .foreachBatch(process_data) \
    .outputMode("append") \
    .start()

#### Capa silver procesamiento
####

# Obtener la sesión de Spark existente si está activa
spark = SparkSession.builder.getOrCreate()

# Función para leer archivos Parquet desde HDFS
def leer_archivos_parquet(path: str) -> DataFrame:
    try:
        # Verificar la existencia del archivo antes de leerlo
        if os.system(f"hdfs dfs -test -e {path}") == 0:
            return spark.read.parquet(path)
        else:
            print(f"El archivo Parquet {path} no existe.")
            return None
    except Exception as e:
        print(f"Error al leer el archivo Parquet {path}: {e}")
        return None
    
# Ruta de bronze
bronze_path = "hdfs:///user/root/bronze"  

# Leer archivos Parquet desde el directorio en HDFS
df_bronze = leer_archivos_parquet(bronze_path)

# Función para agregar una columna con valor constante al precio y dividir la fecha
def transformar_df(df: DataFrame) -> DataFrame:
        df_transformado = df \
            .withColumn("price", lit(3500)) \
            .withColumn("sales", col("quantity_products") * col("price")) \
            .withColumn("commission_value", round(col("sales") * col("employee_commission"), 0)) \
            .withColumn("customer_name", trim(col("customer_name"))) \
            .withColumn("employee_name", trim(col("employee_name"))) \
            .withColumn("commune_name", trim(col("commune_name"))) \
            .withColumn("year", year(col("date"))) \
            .withColumn("month", month(col("date"))) \
            .withColumn("day", dayofmonth(col("date"))) \
            .withColumn("day_week", dayofweek(col("date"))) \
            .withColumn("hour", hour(col("date"))) \
            .withColumn("minute", minute(col("date"))) \
            .withColumn("second", second(col("date")))
        return df_transformado

# Función para guardar DataFrame en un archivo Parquet en la capa Silver, siempre sobreescribe esto es compactar
def guardar_archivo_parquet(df: DataFrame, path: str) -> None:
    df.coalesce(1).write \
        .mode("append") \
        .parquet(path)
        
# Función principal para unir archivos de Bronze a Silver
def unir_archivos_bronze_a_silver(bronze_path: str, silver_path: str) -> None:
    # Leer archivos Parquet desde la capa Bronze
    df_bronze = leer_archivos_parquet(bronze_path)

    # Transformar el DataFrame de Bronze
    df_transformado = transformar_df(df_bronze)

    # Guardar el DataFrame transformado en la capa Silver
    guardar_archivo_parquet(df_transformado, silver_path)

    # Leer archivos Parquet desde la capa Silver después de transformar
    df_silver_transformado = leer_archivos_parquet(silver_path)

    if df_silver_transformado is not None:
        # Contar la cantidad de registros en la capa Silver después de transformar
        records_processed = df_silver_transformado.count()
        print(f"Cantidad de registros procesados en silver después de transformar: {records_processed}")
    
    # Mostrar el DataFrame transformado (opcional)
    #df_transformado.printSchema()
    #df_transformado.show()

# Rutas en hdfs (distrbuido)
bronze_path = "hdfs:///user/root/bronze"  
silver_path = "hdfs:///user/root/silver/unificado.parquet"  

# Ejecutar el proceso de unión
unir_archivos_bronze_a_silver(bronze_path, silver_path)

#### Capa gold
####

# Obtener la sesión de Spark existente si está activa
spark = SparkSession.builder.getOrCreate()

# Ruta del archivo Parquet en la capa Silver
silver_path = "hdfs:///user/root/silver/unificado.parquet"

# Leer los datos desde el archivo Parquet en la capa Silver
df = spark.read.parquet(silver_path)

# Crear la base de datos si no existe
spark.sql("CREATE DATABASE IF NOT EXISTS UNALwater")

# Establecer la base de datos en uso
spark.sql("USE UNALwater")

# Definir la ruta en HDFS donde se guardará la tabla en formato Parquet en la capa Gold
gold_path = "hdfs:///user/root/gold/UNALWater"

# Insertar el dataframe en una tabla externa UNALWater particionada por el campo 'date'
df.write.mode("append") \
  .partitionBy("date") \
  .format("parquet") \
  .option("path", gold_path) \
  .saveAsTable("UNALWater")

# Construir y ejecutar consultas SQL para responder preguntas de negocio
query_ventas_por_comuna = """
    SELECT 
        CASE 
            WHEN commune_name LIKE '%CORREGIMIENTO DE SAN SEBAS%' THEN 'SAN SEBASTIAN DE PALMITAS'
            WHEN commune_name LIKE '%CORREGIMIENTO DE SAN CRIS%' THEN 'SAN CRISTOBAL'
            WHEN commune_name = 'CORREGIMIENTO DE ALTAVISTA' THEN 'ALTAVISTA'
            WHEN commune_name = 'CORREGIMIENTO DE SANTA ELENA' THEN 'SANTA ELENA'
            WHEN commune_name = 'CORREGIMIENTO DE SAN ANTONIO DE PRADO' THEN 'SAN ANTONIO DE PRADO'
        ELSE commune_name END AS Comuna_Corregimiento,
        SUM(quantity_products) AS Cantidad_Productos,
        SUM(sales) AS Total_Ventas
    FROM UNALWater
    GROUP BY Comuna_Corregimiento
    ORDER BY Total_Ventas DESC;
"""

query_ventas_por_vendedor = """
    SELECT 
        employee_name AS Vendedor,
        SUM(quantity_products) AS Cantidad_Productos,
        SUM(sales) AS Total_Ventas,
        SUM(commission_value) AS Valor_Comision
    FROM UNALWater
    GROUP BY employee_name
    ORDER BY Valor_Comision DESC
"""

query_top10_por_clientes = """
    SELECT 
        customer_name AS Cliente,
        SUM(quantity_products) AS Cantidad_Productos,
        SUM(sales) AS Total_Ventas
    FROM UNALWater
    GROUP BY customer_name
    ORDER BY Total_Ventas DESC
    LIMIT 10
"""

query_ventas_por_dia = """
    SELECT 
        CASE 
            WHEN day_week = 1 THEN 'Domingo' 
            WHEN day_week = 2 THEN 'Lunes' 
            WHEN day_week = 3 THEN 'Martes'
            WHEN day_week = 4 THEN 'Miércoles'
            WHEN day_week = 5 THEN 'Jueves'
            WHEN day_week = 6 THEN 'Viernes'
            WHEN day_week = 7 THEN 'Sábado'
        END AS Dia_Semana,
        SUM(quantity_products) AS Cantidad_Productos,
        SUM(sales) AS Total_Ventas
    FROM UNALWater
    GROUP BY day_week
    ORDER BY Total_Ventas DESC
"""

# Ejecutar las consultas SQL y mostrar los resultados
ventas_por_comuna = spark.sql(query_ventas_por_comuna)
print("Comportamiento de la cantidad de ventas por comuna:")
ventas_por_comuna.show()

ventas_por_vendedor = spark.sql(query_ventas_por_vendedor)
print("Comportamiento de la cantidad de ventas por vendedor:")
ventas_por_vendedor.show()

top10_por_clientes = spark.sql(query_top10_por_clientes)
print("Los 10 clientes que más nos han comprado botellas de agua:")
top10_por_clientes.show()

ventas_por_dia = spark.sql(query_ventas_por_dia)
print("Comportamiento de ventas de botellas de agua por día de la semana:")
ventas_por_dia.show()

######################################## GRAFICA ###################################

spark = SparkSession.builder.getOrCreate()

# Definir la ruta del archivo Parquet con datos de ventas en la capa Silver
cargue_inicial_path = 'hdfs:///user/root/silver/unificado.parquet'

try:
    # Cargar el archivo Parquet de cargue inicial como un DataFrame de Spark
    df_cargue = spark.read.parquet(cargue_inicial_path)

    # Convertir las columnas de longitud y latitud a tipo Float
    df_cargue = df_cargue.withColumn('longitude', col('longitude').cast(FloatType())) \
                         .withColumn('latitude', col('latitude').cast(FloatType()))

    # Convertir el DataFrame de Spark en un DataFrame de Pandas
    df_cargue_pd = df_cargue.toPandas()

    # Crear una columna de geometría en el DataFrame de Pandas
    df_cargue_pd['geom'] = df_cargue_pd.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

    # Convertir el DataFrame de Pandas en un GeoDataFrame de GeoPandas
    gdf_cargue = gpd.GeoDataFrame(df_cargue_pd, geometry='geom')
    
    # Cargar el archivo Parquet de geometrías de Medellín como un GeoDataFrame de GeoPandas
    medellin_neighborhoods = gpd.read_parquet('./data/medellin_neighborhoods.parquet')

    # Crear una escala de colores basada en las ventas
    norm = mcolors.Normalize(vmin=gdf_cargue['sales'].min(), vmax=gdf_cargue['sales'].max())
    cmap = cm.ScalarMappable(norm=norm, cmap='viridis')

    # Asignar colores a cada punto basado en las ventas
    gdf_cargue['color'] = gdf_cargue['sales'].apply(lambda x: cmap.to_rgba(x))

    # Crear el gráfico con las geometrías de Medellín y los puntos del cargue inicial
    fig, ax = plt.subplots(figsize=(15, 15))
    medellin_neighborhoods.plot(ax=ax, color='lightgrey', edgecolor='darkblue')
    
    # Graficar los puntos con tamaño proporcional a las ventas y colores variados
    gdf_cargue.plot(ax=ax, color=gdf_cargue['color'], markersize=gdf_cargue['sales'] / 100, alpha=0.6)
    
    plt.title('Datos de ventas en Medellín y ubicaciones de clientes', fontsize = 16)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    plt.gca().axes.get_yaxis().set_visible(False)
    plt.gca().axes.get_xaxis().set_visible(False)
    plt.grid(False)
    
    # Añadir la barra de color
    cax = fig.add_axes([0.95, 0.45, 0.02, 0.3])
    cbar = plt.colorbar(cmap, cax=cax, orientation='vertical', label='Ventas')
    
    # Guardar la figura en un archivo si la visualización es correcta
    plt.savefig('./medellin_neighborhoods_simulacion.png')
    plt.show()
    plt.close()
    
except Exception as e:
    print(f"Error al convertir o visualizar los datos con GeoPandas: {str(e)}")

######################################## MACHINE LEARNING

# Crear sesión de Spark
spark = SparkSession.builder.appName("GeoMLModel").getOrCreate()

# Cargar datos desde archivo Parquet
df = spark.read.parquet("ruta/al/archivo.parquet")

# Seleccionar características relevantes (latitude, longitude) y la variable objetivo (sales)
features = ['latitude', 'longitude']
target = 'sales'

# Filtrar filas con datos nulos en las columnas seleccionadas
df = df.select(*features, target).dropna()

# Ensamblar las características en un vector
assembler = VectorAssembler(inputCols=features, outputCol="features")
df = assembler.transform(df)

# Dividir los datos en conjuntos de entrenamiento y prueba
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Crear y entrenar el modelo de regresión lineal
lr = LinearRegression(labelCol=target, featuresCol="features")
lr_model = lr.fit(train_data)

# Realizar predicciones en el conjunto de prueba
predictions = lr_model.transform(test_data)

# Evaluar el modelo
evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Mostrar las primeras predicciones
predictions.select("features", target, "prediction").show(truncate=100)

query.awaitTermination()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


El archivo Parquet hdfs:///user/root/bronze no existe.


+--------+---------+----+-----------+-----------+-----------------+--------+------------+------------+-------------+-------------+-------------------+
|latitude|longitude|date|customer_id|employee_id|quantity_products|order_id|commune_code|commune_name|customer_name|employee_name|employee_commission|
+--------+---------+----+-----------+-----------+-----------------+--------+------------+------------+-------------+-------------+-------------------+
+--------+---------+----+-----------+-----------+-----------------+--------+------------+------------+-------------+-------------+-------------------+



Cantidad de registros procesados en silver después de transformar: 0
+-----------------+------------------+-------------------+-----------+-----------+-----------------+----------+------------+-------------------------------------+-----------------+-----------------+-------------------+
|latitude         |longitude         |date               |customer_id|employee_id|quantity_products|order_id  |commune_code|commune_name                         |customer_name    |employee_name    |employee_commission|
+-----------------+------------------+-------------------+-----------+-----------+-----------------+----------+------------+-------------------------------------+-----------------+-----------------+-------------------+
|6.251009155186926|-75.60383075677663|2023-06-21 12:23:59|8740       |1114       |46               |6944468493|12          |LA AMÉRICA                           |Christen Hopkins |Bevis Sanford    |0.14               |
|6.213709158410613|-75.64667271349606|2024-01-08 23:51:

Comportamiento de la cantidad de ventas por comuna:
+--------------------+------------------+------------+
|Comuna_Corregimiento|Cantidad_Productos|Total_Ventas|
+--------------------+------------------+------------+
+--------------------+------------------+------------+

Comportamiento de la cantidad de ventas por vendedor:
+--------+------------------+------------+--------------+
|Vendedor|Cantidad_Productos|Total_Ventas|Valor_Comision|
+--------+------------------+------------+--------------+
+--------+------------------+------------+--------------+

Los 10 clientes que más nos han comprado botellas de agua:
+-------+------------------+------------+
|Cliente|Cantidad_Productos|Total_Ventas|
+-------+------------------+------------+
+-------+------------------+------------+

Comportamiento de ventas de botellas de agua por día de la semana:
+----------+------------------+------------+
|Dia_Semana|Cantidad_Productos|Total_Ventas|
+----------+------------------+------------+
+---------

+------------------+------------------+-------------------+-----------+-----------+-----------------+----------+------------+------------------------------------------+---------------+---------------+-------------------+
|latitude          |longitude         |date               |customer_id|employee_id|quantity_products|order_id  |commune_code|commune_name                              |customer_name  |employee_name  |employee_commission|
+------------------+------------------+-------------------+-----------+-----------+-----------------+----------+------------+------------------------------------------+---------------+---------------+-------------------+
|6.246027959840931 |-75.5952699276273 |2023-06-18 10:32:33|3720       |4942       |26               |95339840  |11          |LAURELES ESTADIO                          |Jin Gonzalez   |Sydnee Kirby   |0.16               |
|6.2043447566507925|-75.68564716184336|2024-02-17 12:11:46|9784       |2232       |72               |522505214 |80  

ERROR:root:Exception while sending command.                                     
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answ

Error al procesar los datos: An error occurred while calling o282.showString


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:37263)
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2093/1090041628.py", line 359, in <cell line: 359>
    query.awaitTermination()
  File "/usr/local/lib/python3.8/dist-packages/pyspark/sql/streaming.py", line 101, in awaitTermination
    return self._jsq.awaitTermination()
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.8/dist-packages/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/local/lib/python3.8/dist-packages/py4j/protocol.py", line 334, in get_return_value
    raise Py4JError(
py4j.protocol.Py4JError: An error occurred while calling o66.awaitTermination

During handling of 

Py4JError: An error occurred while calling o66.awaitTermination